## Test insertions

I have sampled local matches with different reads and error rates from `ref.fasta`. I have then inserted these local matches into `query/query.fasta` at random positions to create `query/with_insertions_{er}.fasta`. The file `ground_truth/{er}.tsv` shows the location of each local match in the `query/with_insertions_{er}.fasta`. 

This notebook tests if the ground truth file has the correct locations.  

In [1]:
from Bio import SeqIO
import pandas as pd
import random

Compare the original query file and the query with insertions. Check if matches from ground truth actually appear where they are supposed to.

In [2]:
rep = "2"
er = "01"

original_query_file = "../query/one_line_" + rep + ".fasta"
query_with_insertions_file = "../query/with_insertions_" + rep + "_" + er + ".fasta"
# the ground truth file shows positions in the query with insertions
ground_truth_file = "../ground_truth/" + rep + "_" + er + ".tsv"
local_matches_file = "../local_matches/" + rep + "_" + er + ".fastq"

In [3]:
original_query = list(SeqIO.parse(original_query_file, "fasta"))
query_with_insertions = list(SeqIO.parse(query_with_insertions_file, "fasta"))
local_matches = list(SeqIO.parse(local_matches_file, "fastq"))
ground_truth = pd.read_csv(ground_truth_file, sep='\t')

Take a random local match from the local matches file. Find it's assigned position in the ground truth file and extract that location of the query with insertions file.

In [4]:
random.seed(42)
random_insertion = random.choice(local_matches)
random_insertion

SeqRecord(seq=Seq('ATTCGGTCAAGAACACCCACAGTAATTCCTGGGCGATACCAGACTGTACTAAAT...CCG', SingleLetterAlphabet()), id='l150-77', name='l150-77', description='l150-77', dbxrefs=[])

In [5]:
ground_truth.loc[ground_truth['id'] == random_insertion.name]

,id,position,length
482,l150-77,1073847,150


In [6]:
position = ground_truth.loc[ground_truth['id'] == random_insertion.name]["position"].values[0]
position

1073847

In [7]:
length = ground_truth.loc[ground_truth['id'] == random_insertion.name]["length"].values[0]
length

150

In [8]:
original_insertion = str(random_insertion.seq)
original_insertion

'ATTCGGTCAAGAACACCCACAGTAATTCCTGGGCGATACCAGACTGTACTAAATCTACTGCTCCGTTTAGACAACATCACATCTTAGCTTGTTTTGAGCCGGCATCTCGCTACCACTCCGCTAATACCATTCTCTAGTCTTGTGGTGCCG'

In [9]:
inserted_into_query = str(query_with_insertions[0].seq[position:position + length])
inserted_into_query

'ATTCGGTCAAGAACACCCACAGTAATTCCTGGGCGATACCAGACTGTACTAAATCTACTGCTCCGTTTAGACAACATCACATCTTAGCTTGTTTTGAGCCGGCATCTCGCTACCACTCCGCTAATACCATTCTCTAGTCTTGTGGTGCCG'

In [10]:
assert(original_insertion == inserted_into_query),"Something went wrong with the insertion"

### Check insertions

Does the simulated local match actually appear at the position that is specified in the ground truth file

In [11]:
def check_insertion_correctness():
    random_insertion = random.choice(local_matches)
    position = ground_truth.loc[ground_truth['id'] == random_insertion.name]["position"].values[0]
    length = ground_truth.loc[ground_truth['id'] == random_insertion.name]["length"].values[0]
    original_insertion = str(random_insertion.seq)
    inserted_into_query = str(query_with_insertions[0].seq[position:position + length])
    assert(original_insertion == inserted_into_query),"Something went wrong with the insertion"
    print("Insertion " + random_insertion.name + " found at position " + str(position))

In [12]:
for i in range(10):
    check_insertion_correctness()

Insertion l50-57 found at position 246564
Insertion l50-12 found at position 48787
Insertion l200-4 found at position 530173
Insertion l100-15 found at position 405392
Insertion l100-0 found at position 1105109
Insertion l50-114 found at position 960345
Insertion l50-71 found at position 915344
Insertion l200-2 found at position 207000
Insertion l50-52 found at position 157853
Insertion l150-96 found at position 541964


### Check positions before first insertion

Is the leading sequence (before the first insertion) equal for the original query and the query with insertions?

In [13]:
ground_truth.head()

,id,position,length
0,l50-2,4509,50
1,l100-116,7855,100
2,l200-94,8027,200
3,l200-17,11641,200
4,l150-21,12246,150


In [14]:
first_insertion_position = ground_truth.iloc[0]['position']

str(original_query[0].seq[0:first_insertion_position]) == \
str(query_with_insertions[0].seq[0:first_insertion_position])

True

### Check a sample of positions in the middle

In [15]:
# get random positions from the ground truth list to verify them
ran_ind_list = random.sample(range(len(ground_truth['id'])), 10)
ran_ind_list.sort()

In [16]:
for ind in ran_ind_list:
    if (ind == 0):
        continue
    if (ind == len(ground_truth['id'])):
        continue
    # the ground truth positions represent indices in the query with insertions
    previous_insert_position = ground_truth.iloc[ind-1]['position']
    previous_insert_length = ground_truth.iloc[ind-1]['length']
    downstream_bias = sum(ground_truth.truncate(after=(ind-1))['length'])

    insert_position = ground_truth.iloc[ind]['position']
    insert_length = ground_truth.iloc[ind]['length']
    upstream_bias = sum(ground_truth.truncate(after=(ind))['length'])

    next_insert_position = ground_truth.iloc[ind+1]['position']

    original_query_downstream = str(original_query[0].seq[previous_insert_position - downstream_bias \
                                                          + previous_insert_length \
                                                          :insert_position - downstream_bias])

    insertion_downstream = str(query_with_insertions[0].seq[previous_insert_position + \
                                                            previous_insert_length:insert_position])

    if (original_query_downstream == insertion_downstream):
        print("Verified downstream sequence for insertion " + ground_truth.iloc[ind]['id'])
    else:
        print("Incorrect downstream " + ground_truth.iloc[ind]['id'])

    original_query_upstream = str(original_query[0].seq[insert_position - upstream_bias \
                                                        + insert_length \
                                                        :next_insert_position - upstream_bias])
    insertion_upstream = str(query_with_insertions[0].seq[insert_position + insert_length:next_insert_position])

    if (original_query_upstream == insertion_upstream):
        print("Verified upstream sequence for insertion " + ground_truth.iloc[ind]['id'])
    else:
        print("Incorrect upstream " + ground_truth.iloc[ind]['id'])
    

Verified downstream sequence for insertion l200-114
Verified upstream sequence for insertion l200-114
Verified downstream sequence for insertion l50-92
Verified upstream sequence for insertion l50-92
Verified downstream sequence for insertion l50-106
Verified upstream sequence for insertion l50-106
Verified downstream sequence for insertion l200-76
Verified upstream sequence for insertion l200-76
Verified downstream sequence for insertion l200-47
Verified upstream sequence for insertion l200-47
Verified downstream sequence for insertion l100-58
Verified upstream sequence for insertion l100-58
Verified downstream sequence for insertion l200-72
Verified upstream sequence for insertion l200-72
Verified downstream sequence for insertion l150-94
Verified upstream sequence for insertion l150-94
Verified downstream sequence for insertion l50-53
Verified upstream sequence for insertion l50-53
Verified downstream sequence for insertion l50-17
Verified upstream sequence for insertion l50-17


### Check positions after last insertion

Is the trailing sequence (after the last insertion) equal for the original query and the query with insertions?

In [17]:
ground_truth.tail()

,id,position,length
495,l150-9,1094343,150
496,l50-93,1096555,50
497,l50-113,1097646,50
498,l200-120,1103602,200
499,l100-0,1105109,100


In [18]:
last_insertion_position = ground_truth.iloc[-1]['position']
last_insertion_length = ground_truth.iloc[-1]['length']
bias = sum(ground_truth["length"]) - last_insertion_length

str(original_query[0].seq[last_insertion_position - bias:]) == \
str(query_with_insertions[0].seq[last_insertion_position + last_insertion_length:])

True